# Dependencies

In [40]:
import numpy as np
from tensorflow import keras as k
from tensorflow.keras import layers as l
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks

# Input Data
The XOR problem

In [41]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Hyperparameters

In [42]:
num_samples = len(x)
epochs = 1000

# Regular model

In [43]:
model = k.Sequential()

model.add(l.Dense(units=5, activation='relu', input_dim=2))
model.add(l.Dense(units=7, activation='relu', input_dim=2))
model.add(l.Dense(units=1, activation='sigmoid'))


model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x, y, nb_epoch=epochs, verbose=0)

W0321 19:47:26.048192 4506525120 training.py:792] The `nb_epoch` argument in `fit` has been renamed `epochs`.


# Pruned Model

In [46]:
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0, final_sparsity=0.5, begin_step=1000, end_step=4000)

pruned = tfmot.sparsity.keras.prune_low_magnitude(k.models.clone_model(model), pruning_schedule=pruning_schedule)
pruned.compile(loss='mean_squared_error', optimizer='adam')
pruned.fit(x, y, nb_epoch=epochs, verbose=0, callbacks=[pruning_callbacks.UpdatePruningStep()])

W0321 19:47:41.657231 4506525120 training.py:792] The `nb_epoch` argument in `fit` has been renamed `epochs`.


# Performances

In [47]:
print("Regular model:" )
model_loss = model.evaluate(x,y)
print("~~~~~~~~~~~~~~~")
print("Pruned version:")
pruned_loss = pruned.evaluate(x,y)

Regular model:
4/4 [==============================] - 0s 424us/sample - loss: 0.0076
~~~~~~~~~~~~~~~
Pruned version:
4/4 [==============================] - 0s 17ms/sample - loss: 0.0046
